In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    FunctionTransformer
)
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV
#import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split


In [2]:
#load our data
data = pd.read_csv('data_2016.csv')


In [3]:
#lets train test split
X = data.drop(columns=['bought_highbrow_wines'])
y = data['bought_highbrow_wines']
#lets drop x values whose y is nan
X = X.loc[y.dropna().index]
y = y.dropna()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


KeyError: 'cat_sum'

In [29]:
def final_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Consolidated feature engineering + feature selection.
    creates:
    - Intent signals (wine/premium purchase history)
    - Ability signals (spend capacity)
    - Willingness signals (price sensitivity, discount behavior)
    - Channel readiness (online purchase patterns)
    - Stable context (household, loyalty)
    - Flags for anomalies (SOW_!, negative values)
    """
    df = df.copy()
    
    
    # identify all category columns for aggregations

    cat_cols = [c for c in df.columns if c.startswith("cat_")]
    
    # Coerce all cat_* to numeric
    for c in cat_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)
    

    #  CREATE FLAGS 
    # SOW_! flag: capture outlier indicator before normalizing
    if "SOW_type_colr" in df.columns:
        df["SOW_!"] = (df["SOW_type_colr"] == "!").astype(int)
    
    # Negative value flags for cat_* columns (possible returns/refunds)
    for c in cat_cols:
        if (df[c] < 0).any():
            df[f"{c}_neg_flag"] = (df[c] < 0).astype(int)
    

    #  AGGREGATE FEATURES

    # Total spend (sum of all category purchases)
    df["total_spend"] = df[cat_cols].sum(axis=1)
    
    # Online channel features
    df["n_cogo"] = pd.to_numeric(df.get("n_cogo", 0), errors="coerce").fillna(0)
    df["cogo_rev"] = pd.to_numeric(df.get("cogo_rev", 0), errors="coerce").fillna(0)
    
    df["online_channel_ratio"] = np.where(
        df["total_spend"] > 0,
        df["cogo_rev"] / df["total_spend"],
        0
    )
    
    # Discount behavior
    df["total_discount"] = pd.to_numeric(df.get("total_discount", 0), errors="coerce").fillna(0)
    
    df["discount_ratio"] = np.where(
        df["total_spend"] > 0,
        df["total_discount"] / df["total_spend"],
        0
    )
    
    
    #  WINE AFFINITY FEATURES
    
    # Wine-adjacent premium foods that signal taste alignment
    wine_affinity_cols = [
        "cat_Wijn_Stillewijnen_RAYON",     # Still wines (anchor)
        "cat_Tapas",                      # Wine-paired appetizers
        "cat_KaasSeizoenskazen",           # Seasonal / specialty cheeses
        "cat_VerseKaasFruitkazen",         # Fresh / fruit cheeses
        "cat_VisGerookt",                  # Smoked fish
        "cat_VisVerseSchelpdieren",        # Fresh fish & shellfish
    ]
    wine_affinity_cols = [c for c in wine_affinity_cols if c in df.columns]
    
    df["wine_affinity_spend"] = df[wine_affinity_cols].sum(axis=1) if wine_affinity_cols else 0
    
    df["wine_affinity_ratio"] = np.where(
        df["total_spend"] > 0,
        df["wine_affinity_spend"] / df["total_spend"],
        0
    )
    
    
    #  PREMIUM VS NECESSITY RATIO
    # Premium lifestyle categories (discretionary, taste-driven)
    premium_cols = [
        "cat_Wijn_Stillewijnen_RAYON",   # Premium anchor
        "cat_Tapas",                    # Gourmet food
        "cat_KaasSeizoenskazen",         # Specialty cheese
        "cat_VisGerookt",               # Premium fish
        "cat_Bier_Genietbieren",         # Craft / premium beers
        "cat_Bloemen",                  # Gifting / discretionary
        "cat_ParfumerieEHBO",            # Personal care / premium
        "cat_Textiel_Bedlinnen",         # Lifestyle / home comfort
    ]
    premium_cols = [c for c in premium_cols if c in df.columns]
    
# Necessity categories (survival shopping, family logistics)
    necessity_cols = [
        "cat_Babyluiers",                # Baby diapers
        "cat_Incontinentie_luiers",      # Adult diapers
        "cat_MelkKarnemelk",             # Basic dairy
        "cat_BroodKorthoudbaar",         # Bread staples
        "cat_Bot_Mar_Boter",             # Butter (basic)
    ]
    necessity_cols = [c for c in necessity_cols if c in df.columns]
    
    premium_spend = df[premium_cols].sum(axis=1) if premium_cols else 0
    necessity_spend = df[necessity_cols].sum(axis=1) if necessity_cols else 0
    
# Premium ratio: lifestyle orientation vs survival shopping
    df["premium_ratio"] = np.where(
        (premium_spend + necessity_spend) > 0,
        premium_spend / (premium_spend + necessity_spend),
        0.5  # Neutral if no signal
    )
    
    
    #  CLEAN OTHER NUMERIC FEATURES
    
    df["rev_ticket"] = pd.to_numeric(df.get("rev_ticket", 0), errors="coerce").fillna(0)
    df["prod_ticket"] = pd.to_numeric(df.get("prod_ticket", 0), errors="coerce").fillna(0)
    df["price_sens_colr"] = pd.to_numeric(df.get("price_sens_colr", 0), errors="coerce").fillna(0)
    df["SOW_colr"] = pd.to_numeric(df.get("SOW_colr", 0), errors="coerce").fillna(0)
    
    # Keep wine anchor feature directly
    if "cat_Wijn_Stillewijnen_RAYON" in df.columns:
        df["cat_Wijn_Stillewijnen_RAYON"] = pd.to_numeric(
            df["cat_Wijn_Stillewijnen_RAYON"], errors="coerce"
        ).fillna(0)
    
    
    #  HANDLE CATEGORICAL FEATURES
    
    # HOUSEHOLDTYPOLOGY: normalize "!" to "unknown"
    if "HOUSEHOLDTYPOLOGY" in df.columns:
        df["HOUSEHOLDTYPOLOGY"] = (
            df["HOUSEHOLDTYPOLOGY"]
            .fillna("unknown")
            .replace("!", "unknown")
            .astype(str)
        )
    
    if "SOW_type_colr" in df.columns:
        df["SOW_type_colr"] = (
            df["SOW_type_colr"]
            .fillna("unknown")
            .replace("!", "unknown")
            .astype(str)
        )
    
    
    # SELECT FINAL FEATURE SET
    
    final_numeric = [
        "total_spend",
        "rev_ticket",
        "prod_ticket",
        "n_cogo",
        "cogo_rev",
        "online_channel_ratio",
        "price_sens_colr",
        "discount_ratio",
        "cat_Wijn_Stillewijnen_RAYON",
        "wine_affinity_spend",
        "wine_affinity_ratio",
        "premium_ratio",
        "SOW_colr",
        "SOW_!",  # Added: outlier flag
    ]
    
    # Add any neg_flag columns that were created
    neg_flag_cols = [c for c in df.columns if c.endswith("_neg_flag")]
    final_numeric.extend(neg_flag_cols)
    
    final_categorical = [
        "HOUSEHOLDTYPOLOGY",
        "SOW_type_colr",
    ]
    
    # Only keep columns that exist
    final_numeric = [c for c in final_numeric if c in df.columns]
    final_categorical = [c for c in final_categorical if c in df.columns]
    
    return df[final_numeric + final_categorical], final_numeric, final_categorical

In [30]:
# feature_engineering() is now integrated into final_features()
# This cell kept for reference but function removed to avoid confusion
pass


In [31]:
from sklearn.base import BaseEstimator, TransformerMixin

class ClipLog(BaseEstimator, TransformerMixin):
    """Log1p transform with negative clipping. Supports get_feature_names_out."""
    
    def fit(self, X, y=None):
        # Store feature names if available
        if hasattr(X, 'columns'):
            self._feature_names = list(X.columns)
        elif hasattr(X, 'shape'):
            self._feature_names = [f"x{i}" for i in range(X.shape[1])]
        return self

    def transform(self, X):
        arr = np.array(X, dtype=float, copy=True)
        arr[arr < 0] = 0
        return np.log1p(arr)
    
    def get_feature_names_out(self, input_features=None):
        """Required for sklearn pipeline feature name propagation because we have custom transformer so it is not letting sklearn infer names."""
        if input_features is not None:
            return np.array(input_features)
        return np.array(self._feature_names)

In [19]:
# #making categories 
# # Spend / turnover-like features
# spend_cols = [col for col in X_train.columns if col.startswith("cat_")] + [
#     "cogo_rev",
#     "total_discount",
#     "rev_ticket"
# ]

# # Count-like features (can also be scaled)
# count_cols = [
#     "prod_ticket",
#     "n_cogo"
# ]

# # Other numeric features
# other_numeric_cols = [
#     "price_sens_colr",
#     "SOW_colr"
# ]

# numeric_cols = spend_cols + count_cols + other_numeric_cols

# categorical_cols = [
#     "HOUSEHOLDTYPOLOGY",
#     "SOW_type_colr"
# ]

import numpy as np
import pandas as pd
#we are doing diff transforms based on feature groups binary, log_numeric, categorical
# infer_feature_groups() removed - using explicit feature groups in finalpipeline()
# Explicit groups are preferred for interpretability and control
pass


In [33]:
def finalpipeline(numeric_cols: list, categorical_cols: list):
    """
        
        
    Key design choices:
    - ElasticNet regularization (handles correlated features properly) and chosen for interpretability over black-box models (e.g. tree ensembles despite their slightly higher raw performance)
    - Log-transform for skewed spend features
    - StandardScaler for all numeric (critical for regularization)
    - OneHotEncoder for categoricals
    """
    # Separate log-transform candidates (spend/revenue features) from others
    log_transform_cols = [
        "total_spend",
        "cogo_rev",
        "cat_Wijn_Stillewijnen_RAYON",
        "wine_affinity_spend",
        "rev_ticket",
    ]
    log_transform_cols = [c for c in log_transform_cols if c in numeric_cols]
    
    # Non-log numeric (ratios, counts, scores, flags - already bounded or normalized)
    standard_numeric_cols = [c for c in numeric_cols if c not in log_transform_cols]
    
    # Log-transform pipeline (for skewed spend features)
    log_numeric_pipeline = Pipeline([
        ("impute", SimpleImputer(strategy="constant", fill_value=0)),
        ("clip_log", ClipLog()),
        ("scaler", StandardScaler())
    ])
    
    # Standard numeric pipeline (for ratios, scores, counts, flags)
    standard_numeric_pipeline = Pipeline([
        ("impute", SimpleImputer(strategy="constant", fill_value=0)),
        ("scaler", StandardScaler())
    ])
    
    # Categorical pipeline (sklearn version compatibility)
    try:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)
    
    categorical_pipeline = Pipeline([
        ("impute", SimpleImputer(strategy="constant", fill_value="missing")),
        ("ohe", ohe)
    ])
    
    # Build transformers list dynamically
    transformers = []
    if log_transform_cols:
        transformers.append(("log_num", log_numeric_pipeline, log_transform_cols))
    if standard_numeric_cols:
        transformers.append(("std_num", standard_numeric_pipeline, standard_numeric_cols))
    if categorical_cols:
        transformers.append(("cat", categorical_pipeline, categorical_cols))
    
    preprocessor = ColumnTransformer(
        transformers=transformers,
        remainder="drop",
        verbose_feature_names_out=False
    )
    
    # Full model pipeline with ElasticNet
    model = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("classifier", LogisticRegression(
            penalty="elasticnet",
            solver="saga",          # Only solver that supports elasticnet
            l1_ratio=0.5,           # Balance between L1 and L2
            C=1.0,                  # Regularization strength (will tune)
            max_iter=5000,          # saga needs more iterations
            class_weight="balanced",
            random_state=42
        ))
    ])
    
    feature_groups = {
        "log_numeric": log_transform_cols,
        "standard_numeric": standard_numeric_cols,
        "categorical": categorical_cols,
    }
    
    return model, feature_groups

In [34]:
# Apply feature engineering to train and test sets (ONCE)
X_train_eng, num_cols, cat_cols = final_features(X_train)
X_test_eng, _, _ = final_features(X_test)

# Ensure test has same columns as train (missing columns -> fill with 0)
X_test_eng = X_test_eng.reindex(columns=X_train_eng.columns, fill_value=0)

print(f"Engineered features: {X_train_eng.shape[1]} total")
print(f"  Numeric: {len(num_cols)}")
print(f"  Categorical: {len(cat_cols)}")
print(f"\nNumeric features: {num_cols}")
print(f"Categorical features: {cat_cols}")

# Build the pipeline (pass feature lists, NOT raw data)
gold_model, feature_groups = finalpipeline(num_cols, cat_cols)

# Show feature groups
print("\n" + "="*60)
print("FEATURE GROUPS")
print("="*60)
for k, v in feature_groups.items():
    print(f"\n{k.upper()} ({len(v)}):")
    for c in v:
        print("  ", c)

Engineered features: 47 total
  Numeric: 45
  Categorical: 2

Numeric features: ['total_spend', 'rev_ticket', 'prod_ticket', 'n_cogo', 'cogo_rev', 'online_channel_ratio', 'price_sens_colr', 'discount_ratio', 'cat_Wijn_Stillewijnen_RAYON', 'wine_affinity_spend', 'wine_affinity_ratio', 'premium_ratio', 'SOW_colr', 'SOW_!', 'cat_Babyluiers_neg_flag', 'cat_Ber_Ger_DVPortioneerbaar_neg_flag', 'cat_Ber_Ger_VersMaaltijdsalades_neg_flag', 'cat_Bier_Genietbieren_neg_flag', 'cat_Bot_Mar_Boter_neg_flag', 'cat_BroodKorthoudbaar_neg_flag', 'cat_Chips_neg_flag', 'cat_ColruytMobile_Toestellen_neg_flag', 'cat_EleKtroKeuken_neg_flag', 'cat_Houtpelletskolen_briketten_neg_flag', 'cat_Incontinentie_luiers_neg_flag', 'cat_KaasSeizoenskazen_neg_flag', 'cat_KoudeSauzen_neg_flag', 'cat_MelkKarnemelk_neg_flag', 'cat_Notengedroogdfruit_groenten_neg_flag', 'cat_ParfumerieEHBO_neg_flag', 'cat_Tapas_neg_flag', 'cat_Textiel_Herenondergoed_neg_flag', 'cat_Textiel_Pantys_neg_flag', 'cat_VNCBGBereidegerechten_neg_flag

In [35]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate
from sklearn.metrics import make_scorer, average_precision_score, precision_recall_curve
import warnings
warnings.filterwarnings('ignore')


#  STABILITY CHECK ( fixed params first because if model is not stable, tuning is pointless)


print("STEP 1: STABILITY CHECK (Fixed Parameters)")
print("Running CV with fixed params to check variance before tuning...")

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Use average_precision (better for imbalanced data than F1 becuase it considers ranking along with other metrics)
stability_scores = cross_validate(
    gold_model,
    X_train_eng,
    y_train,
    cv=cv,
    scoring={
        'average_precision': 'average_precision',
        'f1': 'f1',
        'precision': 'precision',
        'recall': 'recall'
    },
    return_train_score=True
)

print(f"\nAverage Precision: {stability_scores['test_average_precision'].mean():.4f} ± {stability_scores['test_average_precision'].std():.4f}")
print(f"F1 Score:          {stability_scores['test_f1'].mean():.4f} ± {stability_scores['test_f1'].std():.4f}")
print(f"Precision:         {stability_scores['test_precision'].mean():.4f} ± {stability_scores['test_precision'].std():.4f}")
print(f"Recall:            {stability_scores['test_recall'].mean():.4f} ± {stability_scores['test_recall'].std():.4f}")

# Check if variance is acceptable (std < 0.05 is good)
ap_std = stability_scores['test_average_precision'].std()
if ap_std < 0.03:
    print("\n Model is VERY STABLE (std < 0.03)")
elif ap_std < 0.05:
    print("\n Model is STABLE (std < 0.05)")
else:
    print(f"\n Model has HIGH VARIANCE (std = {ap_std:.4f})  proceed with caution")

STEP 1: STABILITY CHECK (Fixed Parameters)
Running CV with fixed params to check variance before tuning...

Average Precision: 0.4097 ± 0.0102
F1 Score:          0.3124 ± 0.0020
Precision:         0.1924 ± 0.0012
Recall:            0.8290 ± 0.0100

 Model is VERY STABLE (std < 0.03)

Average Precision: 0.4097 ± 0.0102
F1 Score:          0.3124 ± 0.0020
Precision:         0.1924 ± 0.0012
Recall:            0.8290 ± 0.0100

 Model is VERY STABLE (std < 0.03)


In [36]:

#  COEFFICIENT STABILITY ANALYSIS(we are checking if coeffs are stable across folds by training model on each fold and collecting coeffs) 
#if model is not stable, we cannot interpret coeffs 

print("STEP 2: COEFFICIENT STABILITY ACROSS FOLDS")


from sklearn.base import clone
#we have this function to get feature names from pipeline becuase we have custom transformers so sklearn cannot infer names automatically
def get_feature_names(model, feature_groups):
    """Simple helper to get feature names from our pipeline."""
    names = feature_groups['log_numeric'] + feature_groups['standard_numeric']
    # Add OHE categories because they expand into multiple features so we need to get their names
    ohe = model.named_steps['preprocess'].named_transformers_['cat'].named_steps['ohe']
    for col_idx, col in enumerate(feature_groups['categorical']):
        for cat in ohe.categories_[col_idx]:
            names.append(f"{col}_{cat}")
    return names

# Collect coefficients from each fold
fold_coefs = []
fold_feature_names = None

for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_train_eng, y_train)):
    X_fold_train = X_train_eng.iloc[train_idx]
    y_fold_train = y_train.iloc[train_idx]
    
    fold_model = clone(gold_model)
    fold_model.fit(X_fold_train, y_fold_train)
    fold_coefs.append(fold_model.named_steps['classifier'].coef_[0])
    
    if fold_feature_names is None:
        fold_feature_names = get_feature_names(fold_model, feature_groups)

fold_coefs = np.array(fold_coefs)

# Check sign stability (because features with signs that change a lot indicate instability)
sign_stability = (np.sign(fold_coefs) == np.sign(fold_coefs.mean(axis=0))).mean(axis=0)
unstable_mask = sign_stability < 0.8

print(f"\nTotal features: {len(fold_feature_names)}")
print(f"Stable features (same sign ≥80% folds): {(~unstable_mask).sum()}")
print(f"Unstable features (sign flips): {unstable_mask.sum()}")

if unstable_mask.any():
    print("\n Unstable features:")
    for name, stab in zip(fold_feature_names, sign_stability):
        if stab < 0.8:
            print(f"   {name}: stable in {stab*100:.0f}% of folds")
else:
    print("\n All features have stable coefficient signs!")

# Top features by mean absolute coefficient (because magnitude matters for impact)
print("\n" + "-"*60)
print("TOP 10 FEATURES BY MEAN |COEFFICIENT|")
print("-"*60)
mean_coefs = fold_coefs.mean(axis=0)
std_coefs = fold_coefs.std(axis=0)
top_idx = np.argsort(np.abs(mean_coefs))[-10:][::-1] 

for idx in top_idx:
    sign = "+" if mean_coefs[idx] > 0 else "-"
    print(f"  {sign} {fold_feature_names[idx]:40s} coef={mean_coefs[idx]:+.3f} ± {std_coefs[idx]:.3f}")

STEP 2: COEFFICIENT STABILITY ACROSS FOLDS

Total features: 71
Stable features (same sign ≥80% folds): 67
Unstable features (sign flips): 4

 Unstable features:
   SOW_!: stable in 60% of folds
   cat_Bier_Genietbieren_neg_flag: stable in 60% of folds
   HOUSEHOLDTYPOLOGY_f_HHnochild_35_54: stable in 40% of folds
   HOUSEHOLDTYPOLOGY_k_HHchild_oldest_13_17: stable in 40% of folds

------------------------------------------------------------
TOP 10 FEATURES BY MEAN |COEFFICIENT|
------------------------------------------------------------
  + rev_ticket                               coef=+1.981 ± 0.019
  + wine_affinity_spend                      coef=+1.762 ± 0.047
  - SOW_type_colr_SOW00-10                   coef=-1.331 ± 0.052
  + SOW_type_colr_Outlier_fr                 coef=+1.301 ± 0.068
  - discount_ratio                           coef=-1.101 ± 0.091
  - SOW_type_colr_SOW10-20                   coef=-1.053 ± 0.048
  - total_spend                              coef=-1.037 ± 0.039
 

In [37]:

# STEP 3: SMALL HYPOTHESIS-DRIVEN GRID SEARCH

print("STEP 3: HYPOTHESIS-DRIVEN TUNING")
#we didnt do gridsearch cv over large grid becuase it can lead to overfitting on cv folds and also we have stability check already done
# small grid, hypothesis-driven, not slot machine

from sklearn.model_selection import GridSearchCV

# Small, focused grid (not 200 combinations as it leads to overfitting on CV folds)
# hypothesis: ElasticNet with moderate regularization should work best
param_grid = {
    'classifier__C': [0.1, 0.5, 1.0, 2.0],      # Regularization strength (small c = strong regularization, simple model, we used 1.0 as default)
    'classifier__l1_ratio': [0.3, 0.5, 0.7],    # L1 vs L2 balance (bigger ratio => many useless features zeroed out, we used 0.5 as default)
}

print(f"\nGrid size: {len(param_grid['classifier__C']) * len(param_grid['classifier__l1_ratio'])} combinations")
print("Scoring: average_precision (better for imbalanced marketing problems)")

grid_search = GridSearchCV(
    estimator=gold_model,
    param_grid=param_grid,
    scoring='average_precision',  #  choice for imbalanced data as it considers ranking
    cv=cv,
    n_jobs=-1,
    verbose=1,
    return_train_score=True
)

grid_search.fit(X_train_eng, y_train)

print(f"\nBest Average Precision: {grid_search.best_score_:.4f}")
print(f"   Best C: {grid_search.best_params_['classifier__C']}")
print(f"   Best l1_ratio: {grid_search.best_params_['classifier__l1_ratio']}")

# Compare default vs tuned
baseline_ap = stability_scores['test_average_precision'].mean()
tuned_ap = grid_search.best_score_
improvement = (tuned_ap - baseline_ap) / baseline_ap * 100

print(f"\n Improvement from tuning: {improvement:+.2f}%")
if improvement < 2:
    print("   → Minimal improvement. Default params are fine.")
elif improvement < 5:
    print("   → Modest improvement. Use tuned params.")
else:
    print("   → Significant improvement. Tuning was worthwhile.")

STEP 3: HYPOTHESIS-DRIVEN TUNING

Grid size: 12 combinations
Scoring: average_precision (better for imbalanced marketing problems)
Fitting 5 folds for each of 12 candidates, totalling 60 fits

Best Average Precision: 0.4097
   Best C: 2.0
   Best l1_ratio: 0.3

 Improvement from tuning: +0.01%
   → Minimal improvement. Default params are fine.

Best Average Precision: 0.4097
   Best C: 2.0
   Best l1_ratio: 0.3

 Improvement from tuning: +0.01%
   → Minimal improvement. Default params are fine.


In [41]:

# STEP 4: THRESHOLD TUNING (we tune threshold, not just weights as we are dealing with probabilities)

from sklearn.metrics import precision_recall_curve, classification_report, confusion_matrix

print("STEP 4: THRESHOLD TUNING")



# Get best model and predict probabilities
best_model = grid_search.best_estimator_
y_proba = best_model.predict_proba(X_test_eng)[:, 1]

# Calculate precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_proba)

# Find optimal threshold for different business objectives
print("\nThreshold Analysis:")
print("-"*60)

#  Maximize F1 , becuase balnces precision and recall
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)
best_f1_idx = np.argmax(f1_scores[:-1])  # exclude last element
best_f1_threshold = thresholds[best_f1_idx]
print(f"Max F1 threshold:           {best_f1_threshold:.3f} (F1={f1_scores[best_f1_idx]:.3f})")

# Recall @ Precision >= 0.7 (realistic for marketing, as good precision to avoid waste)
mask_p70 = precision[:-1] >= 0.7
if mask_p70.any():
    best_recall_at_p70 = recall[:-1][mask_p70].max()
    idx_p70 = np.where((recall[:-1] == best_recall_at_p70) & mask_p70)[0][0]
    threshold_p70 = thresholds[idx_p70]
    print(f"Max Recall @ Precision≥70%: {threshold_p70:.3f} (Recall={best_recall_at_p70:.3f})")
else:
    print("Max Recall @ Precision≥70%: Not achievable")
# Recall @ Precision >= 0.5 (balanced)
mask_p50 = precision[:-1] >= 0.5
if mask_p50.any():
    best_recall_at_p50 = recall[:-1][mask_p50].max()
    idx_p50 = np.where((recall[:-1] == best_recall_at_p50) & mask_p50)[0][0]
    threshold_p50 = thresholds[idx_p50]
    print(f"Max Recall @ Precision≥50%: {threshold_p50:.3f} (Recall={best_recall_at_p50:.3f})")
else:
    print("Max Recall @ Precision≥50%: Not achievable")

# Objective 4: Recall @ Precision >= 0.3 (broader reach)
mask_p30 = precision[:-1] >= 0.3
if mask_p30.any():
    best_recall_at_p30 = recall[:-1][mask_p30].max()
    idx_p30 = np.where((recall[:-1] == best_recall_at_p30) & mask_p30)[0][0]
    threshold_p30 = thresholds[idx_p30]
    print(f"Max Recall @ Precision≥30%: {threshold_p30:.3f} (Recall={best_recall_at_p30:.3f})")
else:
    print("Max Recall @ Precision≥30%: Not achievable")

# Use best F1 threshold for final evaluation as default (we can be changed based on business needs)
CHOSEN_THRESHOLD = best_f1_threshold
print(f"\nUsing threshold: {CHOSEN_THRESHOLD:.3f} (Max F1 objective)")

STEP 4: THRESHOLD TUNING

Threshold Analysis:
------------------------------------------------------------
Max F1 threshold:           0.829 (F1=0.439)
Max Recall @ Precision≥70%: 0.986 (Recall=0.126)
Max Recall @ Precision≥50%: 0.914 (Recall=0.328)
Max Recall @ Precision≥30%: 0.722 (Recall=0.628)

Using threshold: 0.829 (Max F1 objective)


In [42]:

# STEP 5: FINAL TEST SET EVALUATION


print("STEP 5: FINAL TEST SET EVALUATION")


# Predictions with tuned threshold
y_pred_tuned = (y_proba >= CHOSEN_THRESHOLD).astype(int)
y_pred_default = best_model.predict(X_test_eng)  # default 0.5 threshold

print("\nClassification Report (Tuned Threshold):")
print(classification_report(y_test, y_pred_tuned, target_names=["No Highbrow", "Highbrow"]))

print("\nConfusion Matrix (Tuned Threshold):")
cm = confusion_matrix(y_test, y_pred_tuned)
tn, fp, fn, tp = cm.ravel()
print(f"                    Predicted")
print(f"                No         Yes")
print(f"Actual No    {tn:5d}       {fp:5d}")
print(f"Actual Yes   {fn:5d}       {tp:5d}")

print("\n Business Metrics:")
print(f"   True Positives (correctly identified buyers): {tp}")
print(f"   False Positives (wasted marketing): {fp}")
print(f"   False Negatives (missed buyers): {fn}")
print(f"   True Negatives (correctly ignored): {tn}")

# Marketing efficiency 
if tp + fp > 0:
    precision_val = tp / (tp + fp)
    print(f"\n    If you target {tp + fp} customers:")
    print(f"      → {tp} will buy ({precision_val*100:.1f}% hit rate)")
    print(f"      → {fp} won't buy (wasted effort)")

# Compare default vs tuned threshold
print("\n" + "-"*60)
print(" THRESHOLD COMPARISON:")
print("-"*60)

cm_default = confusion_matrix(y_test, y_pred_default)
tn_d, fp_d, fn_d, tp_d = cm_default.ravel()

print(f"Default (0.50): Precision={tp_d/(tp_d+fp_d):.3f}, Recall={tp_d/(tp_d+fn_d):.3f}, TP={tp_d}, FP={fp_d}")
print(f"Tuned ({CHOSEN_THRESHOLD:.2f}):   Precision={precision_val:.3f}, Recall={tp/(tp+fn):.3f}, TP={tp}, FP={fp}")

STEP 5: FINAL TEST SET EVALUATION

Classification Report (Tuned Threshold):
              precision    recall  f1-score   support

 No Highbrow       0.97      0.96      0.97     38008
    Highbrow       0.39      0.50      0.44      1992

    accuracy                           0.94     40000
   macro avg       0.68      0.73      0.70     40000
weighted avg       0.94      0.94      0.94     40000


Confusion Matrix (Tuned Threshold):
                    Predicted
                No         Yes
Actual No    36486        1522
Actual Yes    1003         989

 Business Metrics:
   True Positives (correctly identified buyers): 989
   False Positives (wasted marketing): 1522
   False Negatives (missed buyers): 1003
   True Negatives (correctly ignored): 36486

    If you target 2511 customers:
      → 989 will buy (39.4% hit rate)
      → 1522 won't buy (wasted effort)

------------------------------------------------------------
 THRESHOLD COMPARISON:
-------------------------------------

In [43]:

# STEP 6: FEATURE IMPORTANCE 


print("STEP 6: FEATURE IMPORTANCE INTERPRETATION")


# Get feature names and coefficients
feature_names = get_feature_names(best_model, feature_groups)
coefs = best_model.named_steps['classifier'].coef_[0]

print(f"\nTotal features: {len(coefs)}")

# Create sorted DataFrame
coef_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefs,
}).assign(abs_coef=lambda x: np.abs(x.coefficient)).sort_values('abs_coef', ascending=False)

print("\n TOP 15 FEATURES BY IMPORTANCE:")
print("-"*60)
for _, row in coef_df.head(15).iterrows():
    direction = "↑" if row['coefficient'] > 0 else "↓"
    print(f"  {direction} {row['feature']:40s} {row['coefficient']:+.4f}")

print("\n INTERPRETATION:")
print(" Positive = INCREASES likelihood of buying highbrow wine")
print("Negative = DECREASES likelihood of buying highbrow wine")

print("\n KEY POSITIVE DRIVERS:")
for f in coef_df[coef_df['coefficient'] > 0].head(5)['feature']:
    print(f"   • {f}")

print("\n KEY NEGATIVE DRIVERS:")
for f in coef_df[coef_df['coefficient'] < 0].head(5)['feature']:
    print(f"   • {f}")

STEP 6: FEATURE IMPORTANCE INTERPRETATION

Total features: 71

 TOP 15 FEATURES BY IMPORTANCE:
------------------------------------------------------------
  ↑ rev_ticket                               +1.9807
  ↑ wine_affinity_spend                      +1.7631
  ↓ SOW_type_colr_SOW00-10                   -1.3381
  ↑ SOW_type_colr_Outlier_fr                 +1.3056
  ↓ discount_ratio                           -1.0945
  ↓ SOW_type_colr_SOW10-20                   -1.0591
  ↓ total_spend                              -1.0371
  ↓ prod_ticket                              -0.8288
  ↓ SOW_type_colr_SOW20-30                   -0.7674
  ↓ wine_affinity_ratio                      -0.6687
  ↑ SOW_type_colr_Outlier_om                 +0.6647
  ↓ HOUSEHOLDTYPOLOGY_i_HHchild_oldest_0_5   -0.5797
  ↓ HOUSEHOLDTYPOLOGY_a_Single_18_34         -0.4993
  ↑ HOUSEHOLDTYPOLOGY_e_HHnochild_18_34      +0.4738
  ↓ SOW_type_colr_SOW30-40                   -0.4629

 INTERPRETATION:
 Positive = INCREASES likelihoo

In [ ]:
#Using 2017 datset
# language: python
# filepath: c:\Users\srd9cxl\OneDrive - Colruyt Group NV\Desktop\highbrow wines case\Highbrow-wines-case-predictive-modelling-\entirepipieline.ipynb

# 1.inference data
inference_df = pd.read_csv("C:\\Users\\srd9cxl\\OneDrive - Colruyt Group NV\\Desktop\\highbrow wines case\\Highbrow-wines-case-predictive-modelling-\\data_2017_new.csv")

# 2. Feature engineering
X_inf_eng, _, _ = final_features(inference_df)

# 3. Align columns with training data
X_inf_eng = X_inf_eng.reindex(columns=X_train_eng.columns, fill_value=0)

# 4. Predict probabilities and classes
proba = best_model.predict_proba(X_inf_eng)[:, 1]
pred_class = (proba >= CHOSEN_THRESHOLD).astype(int)

#attaching predictions to inference_df
inference_df["predicted_prob"] = proba
inference_df["bought_highbrow_wines"] = pred_class

# 6. Save or inspect results
inference_df.to_csv("2017_with_predictions.csv", index=False)
print(inference_df[["predicted_prob", "bought_highbrow_wines"]].head())

   predicted_prob  bought_highbrow_wines
0        0.004811                      0
1        0.702256                      0
2        0.080378                      0
3        0.466862                      0
4        0.071346                      0


Fitting 5 folds for each of 16 candidates, totalling 80 fits



KeyboardInterrupt



1.4.2
